In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [27]:
#Read in data from CSV files.

#features = pd.read_csv('clean_data/features_encoded.csv')

#cost_codes = pd.read_csv('clean_data/cost_code_encoded.csv')

df = pd.read_csv('clean_data/clean_data.csv')

df.head()


,Description,Units,Unit Cost,Cost,Cost Code,Vendor_1110438 B.C. Ltd,Vendor_1ST-R8 Welding & Fabrication Ltd.,Vendor_4s Scaffolding,Vendor_596143 BC LTD DBA AVANTE 2000,Vendor_7 Star Security Services Inc,...,Unit of Measure_km,Unit of Measure_kw,Unit of Measure_l,Unit of Measure_m,Unit of Measure_m2,Unit of Measure_m3,Unit of Measure_m3,Unit of Measure_mL,Unit of Measure_t,Cost Code Encoded
0,S&I railing as per quote Aug. 13 2015,0.000000,0.000000,0.218551,05-52-20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,87
1,S&I metal ramp,0.000000,0.000000,0.347495,05-52-20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,87
2,Painting of two offices,0.000000,0.000000,0.393391,09-91-40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,142
3,Demolition - wall/carpeting/platform,0.000000,0.000000,0.328700,09-21-17,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,131
4,Hollow metal frames,0.062893,0.876712,0.139872,08-11-13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,112


In [45]:
cost_codes = df['Cost Code Encoded']
features = df.drop(['Cost Code','Cost Code Encoded'], axis=1)


,Description,Units,Unit Cost,Cost,Vendor_1110438 B.C. Ltd,Vendor_1ST-R8 Welding & Fabrication Ltd.,Vendor_4s Scaffolding,Vendor_596143 BC LTD DBA AVANTE 2000,Vendor_7 Star Security Services Inc,Vendor_A & A Testing Ltd.,...,Unit of Measure_kg,Unit of Measure_km,Unit of Measure_kw,Unit of Measure_l,Unit of Measure_m,Unit of Measure_m2,Unit of Measure_m3,Unit of Measure_m3,Unit of Measure_mL,Unit of Measure_t
0,S&I railing as per quote Aug. 13 2015,0.000000,0.000000,0.218551,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,S&I metal ramp,0.000000,0.000000,0.347495,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Painting of two offices,0.000000,0.000000,0.393391,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Demolition - wall/carpeting/platform,0.000000,0.000000,0.328700,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hollow metal frames,0.062893,0.876712,0.139872,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
#Use sklearn train test split to split the data into training and testing sets. 
#Testing set is 20% of total dataset size.
#Stratify the data so we don't introduce bias in the sets.

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    cost_codes,
                                                    test_size = 0.2,
                                                    stratify = cost_codes
                                                   )



In [50]:
print(X_train.shape)
print(y_train.shape)

(22701, 488)
(22701,)


In [63]:
#Split out X_train and X_test Descriptions for use in sepearate model.
X_train_desc = X_train['Description'].copy()
X_train = X_train.drop('Description', axis=1)

X_test_desc = X_test['Description'].copy()
X_test = X_test.drop('Description', axis=1)

In [64]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter = 5, random_state=42, tol = 1e-3)),
               ])
sgd.fit(X_train_desc, y_train)


y_pred = sgd.predict(X_test_desc)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.41402396053558843
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.67      0.96      0.79       112
           3       0.24      0.50      0.32         8
           4       0.68      0.45      0.54        29
           5       0.73      0.84      0.78        19
           6       0.80      0.80      0.80         5
           7       0.50      0.05      0.08        22
           9       0.75      0.60      0.67         5
          10       0.00      0.00      0.00         1
          11       0.53      0.37      0.44        46
          12       0.53      0.89      0.67       156
          13       0.00      0.00      0.00         1
          14       0.50      0.41      0.45        29
          15       0.00      0.00      0.00         2
          16       0.19      0.25      0.21        12
          17       0.55      0.31      0.40        51
          18       0.53      0.82      0.64       21

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\li